In [3]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pprint

es = Elasticsearch(["13.125.246.197", "52.79.215.253", "52.79.250.228"],
                   http_auth=('elastic', 'rkwjs12#'),
                   port=9200)
es.info()
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
res = es.search(index="iot-skmagic", body={'from':0, 
                                        'size': 1, 
                                        "query": {
                                            "match_all":{}
                                         }
                                        })

pp.pprint(res)

In [ ]:
res = es.delete_by_query(index="iot-skmagic", body={
                                           'query':{"range": 
                                               {
                                                  "offset": {
                                                    "gt": "1338851",
                                                    "lte": "1339851"
                                                  }
                                                }
                                           }
})
pp.pprint(res)

In [ ]:
res = es.search(index="iot-skmagic", body={
    'size': 10000,
    'query': {
        "match_all": {}
    }
})
print(len(res['hits']['hits']))
# pp.pprint(res['hits']['hits'])

In [4]:
from elasticsearch import client
from datetime import datetime
import time
st = datetime.now()
body = {
    "query": {
        "match_all": {}
    },
    "sort": ["_doc"]
}
s = es.count(index='iot-skmagic')['count']
for i in range(0,200):
    es = Elasticsearch(["13.125.246.197", "52.79.215.253", "52.79.250.228"],
                       http_auth=('elastic', 'rkwjs12#'),
                       port=9200)
    scroller = helpers.scan(client=es, query=body, index='iot-skmagic') 
    at = datetime.now()
    a = es.count(index='iot-skmagic')['count']
    print("Before :", a)
    c = 1
    NUMBER_OF_DELETION = 50000
    id_list = []
    body2 = {
        'query': {
            'terms': {
                "_id": id_list
            }
        }
    }
    for sc in scroller:
        if c >= NUMBER_OF_DELETION:
            break
        elif c % (NUMBER_OF_DELETION/2) == 0:
            print(c, " Search ~", datetime.now() - st)
        # pp.pprint(sc)
        # es.delete(index='iot-skmagic', id=sc['_id'])
        id_list.append(sc['_id'])
        c += 1
    # time.sleep(10)
    # while len(id_list) != NUMBER_OF_DELETION-1
    res = es.delete_by_query(index='iot-skmagic', body=body2, wait_for_completion=False)
    print("Time : ", datetime.now() - at, "Delete Count : ", c)
    client.indices.IndicesClient.refresh(self=es,index='iot-skmagic')
    while 1:
        time.sleep(1)
        res = es.count(index="iot-skmagic")['count']
        if NUMBER_OF_DELETION-1-(a-res) == 0:
            break
    print(">> After : ", i, a, res, NUMBER_OF_DELETION-1-(a-res))
print(">> END :", s, res, datetime.now() - st)

Before : 446147015
25000  Search ~ 0:00:02.004347
Time :  0:00:04.088918 Delete Count :  50000
>> After :  0 446147015 446097016 0
Before : 446097016
25000  Search ~ 0:00:25.599279
Time :  0:00:04.632111 Delete Count :  50000
>> After :  1 446097016 446047017 0
Before : 446047017
25000  Search ~ 0:00:51.620423
Time :  0:00:04.139364 Delete Count :  50000
>> After :  2 446047017 445997018 0
Before : 445997018
25000  Search ~ 0:01:18.112013
Time :  0:00:04.271879 Delete Count :  50000
>> After :  3 445997018 445947019 0
Before : 445947019
25000  Search ~ 0:01:44.645834
Time :  0:00:04.524694 Delete Count :  50000
>> After :  4 445947019 445897020 0
Before : 445897020
25000  Search ~ 0:02:11.495841
Time :  0:00:05.859495 Delete Count :  50000


KeyboardInterrupt: 

In [161]:
res = es.count(index="iot-skmagic")['count']
print(a, res, NUMBER_OF_DELETION-1-(a-res))

447546987 447446989 -49999
